In [ ]:
# from torch.utils.data import Dataset, IterableDataset
from datasets import load_dataset, Features
# from transformers import GPT2Tokenizer, GPT2TokenizerFast
# import torch
from tqdm import tqdm
import random
import json
# from tokenizer import TiktokenTokenizer
from gpt import GPTRewardModel, GPT
from configs import get_configs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import tiktoken
config = get_configs('gpt2-medium')



/home/colder/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
path = ''
model = GPT.from_checkpoint(config, path)

In [ ]:
def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode

def generate_gpt2(model, prompt, device, samples=2):
    model.eval()
    model.to(device)
    max_new_tokens = 50
    temperature = 0.1
    top_k = 50
    x, decode = prepare_gpt2_input(prompt, device)

    for k in range(samples):
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        print(decode(y[0].tolist()))
        print('---------------')

In [ ]:
# dataset = load_dataset("xinlai/Math-Step-DPO-10K", split='train')
#take all prompt and chosen from dataset
# Create new dataset with modified values
dataset = dataset.map(lambda x: {'rejected': generate_gpt2(model, x['prompt'], device, samples=1)}, batched=True)

print(dataset[0]['rejected'])

Map: 100%|██████████| 10795/10795 [00:00<00:00, 15568.01 examples/s]

None
